### 기본 디렉토리 설정

In [11]:
import os

# 기본 경로 설정
root_path = '/data/ephemeral/home'
dir_name = 'dataset_yolo'

# 이미지와 레이블의 서브 디렉토리 경로 설정
image_dirs = ['images/train', 'images/valid', 'images/test']
label_dirs = ['labels/train', 'labels/valid']

# 모든 필요한 디렉토리 생성
for dir in image_dirs + label_dirs:
    os.makedirs(os.path.join(root_path, dir_name, dir), exist_ok=True)

### 기존 파일을 욜로 폴더로 옮기기

In [12]:
import shutil
import os
import glob

# 기존 및 새 디렉토리 경로 설정
src_train_images = os.path.join(root_path, 'dataset/train')
src_test_images = os.path.join(root_path, 'dataset/test')
dest_train_images = os.path.join(root_path, dir_name, 'images/train')
dest_test_images = os.path.join(root_path, dir_name, 'images/test')
src_train_json = os.path.join(root_path, 'dataset/train.json')
src_test_json = os.path.join(root_path, 'dataset/test.json')
dest_json = os.path.join(root_path, dir_name)

# train 이미지 복사
for file in glob.glob(f'{src_train_images}/*.jpg'):
    shutil.copy(file, dest_train_images)

# test 이미지 복사
for file in glob.glob(f'{src_test_images}/*.jpg'):
    shutil.copy(file, dest_test_images)

# JSON 파일 복사
shutil.copy(src_train_json, dest_json)
shutil.copy(src_test_json, dest_json)

'/data/ephemeral/home/dataset_yolo/test.json'

### 욜로 폴더 내 JSON 파일을 기반으로 txt 파일 생성

- 주의: 생성된 파일이 바로 안보일 수 있음

In [15]:
train_json_path = '/data/ephemeral/home/dataset_yolo/train.json'
label_path = '/data/ephemeral/home/dataset_yolo/labels'

!python coco2yolo.py -j {train_path} -o {label_path}

total images 4883
total categories 10
total labels 23144
loading image info...
loading done, total images 4883
start converting...
converting done, total labels 4883
saving txt file...
0 [('train/0000.jpg', 0, (0.46044921875, 0.41850585937499996, 0.5349609375, 0.45869140625))]
1 [('train/0001.jpg', 3, (0.028125, 0.48603515625, 0.05625, 0.1763671875)), ('train/0001.jpg', 7, (0.07060546875, 0.53359375, 0.1412109375, 0.17734375)), ('train/0001.jpg', 4, (0.8393066406249999, 0.42905273437499997, 0.26787109375, 0.24599609375)), ('train/0001.jpg', 5, (0.45903320312499996, 0.70576171875, 0.22822265625, 0.1009765625)), ('train/0001.jpg', 5, (0.38359375, 0.55615234375, 0.7599609375, 0.236328125)), ('train/0001.jpg', 0, (0.52099609375, 0.7537109375, 0.211328125, 0.1755859375)), ('train/0001.jpg', 7, (0.158203125, 0.613525390625, 0.1359375, 0.05185546875)), ('train/0001.jpg', 0, (0.643359375, 0.71640625, 0.07109375, 0.0919921875))]
2 [('train/0002.jpg', 3, (0.57001953125, 0.41181640625, 0.61679687

### 욜로 폴더 내 train 파일을 valid로 나눔
- yolo에서 자체적으로 train set 안에 있는 임의로 validation set으로 나누는 기능이 존재함.
- 만약, 위 기능을 사용하려면 아래 코드는 실행시키지 말기

In [ ]:
from sklearn.model_selection import train_test_split

# 이미지 파일이 있는 디렉토리
train_dir = 'dataset_yolo/images/train'

# 검증 세트 이미지가 저장될 디렉토리
valid_dir = 'dataset_yolo/images/valid'

# 훈련 디렉토리에서 이미지 파일 목록 가져오기
images = glob.glob(f'{train_dir}/*.jpg')

# 이미지 파일을 훈련 세트와 검증 세트로 나누기
_, valid_images = train_test_split(images, test_size=0.2, random_state=42)

# 파일을 검증 디렉토리로 이동시키는 함수
def move_files(files, dest_dir):
    for file in files:
        # 이미지 이동
        shutil.move(file, dest_dir)

        # 동일한 이름의 .txt 파일 이동
        txt_file = os.path.splitext(file)[0] + '.txt'
        if os.path.exists(txt_file):
            shutil.move(txt_file, dest_dir)

# 검증 세트 이동
move_files(valid_images, valid_dir)